In [1]:
import numpy as np
from pathlib import Path
from pprint import pprint

from apollo.crosssections import create_crosssection_catalog

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
catalog = create_crosssection_catalog("/Volumes/ResearchStorage/Opacities_0v10/gases")

/Users/arthur/Documents/Astronomy/2019/Retrieval/Code/APOLLO/apollo/crosssections.py:114: UserWarning: There is apparently more than one set of header info. Check to see all have the same pressure, temperature, and wavelength ranges, and/or resolutions.
  warn(


In [4]:
pprint(catalog["nir"])

CrossSectionDirectory(number_of_pressure_layers=18,
                      minimum_log_pressure=-3,
                      maximum_log_pressure=5.5,
                      number_of_temperatures=36,
                      minimum_log_temperature=1.87506,
                      maximum_log_temperature=3.59985,
                      number_of_spectral_elements=21204,
                      minimum_wavelength=0.6,
                      maximum_wavelength=5,
                      effective_resolution=10000,
                      pressures=array([-3. , -2.5, -2. , -1.5, -1. , -0.5,  0. ,  0.5,  1. ,  1.5,  2. ,
        2.5,  3. ,  3.5,  4. ,  4.5,  5. ,  5.5]),
                      temperatures=array([  74.99978182,   84.01168867,   94.1064582 ,  105.41420623,
        118.08068317,  132.26915267,  148.16249601,  165.9655693 ,
        185.90784398,  208.2463646 ,  233.26906191,  261.2984642 ,
        292.69585446,  327.8659271 ,  367.26200429,  411.39187895,
        460.82435996,  516.19660378,  

In [ ]:
test_directory = catalog["nir"]

In [ ]:
opacity_Dataset = load_crosssections_into_Dataset(test_directory)

In [ ]:
opacity_Dataset.wavelengths.attrs.update(dict(units="micron"))

In [ ]:
opacity_Dataset.pressures.attrs.update(dict(units="dex(mbar)"))

In [ ]:
opacity_Dataset.temperatures.attrs.update(dict(units="K"))

In [ ]:
#nc_save_file = "nir_opacities.nc"
#opacity_Dataset.to_netcdf(nc_save_file)

In [ ]:
opacity_Dataset

In [ ]:
from scipy.interpolate import PchipInterpolator as monotonic_interpolation
from scipy.ndimage import gaussian_filter1d as gaussian_smoothing

In [ ]:
def piette(T_m4, T_m3, T_m2, T_m1, T_0, T_0p5, T_1, T_1p5, T_2, T_2p5,
           num_layers_final=71, P_min=-4, P_max=2.5):
    logP_nodes = np.array([-4, -3, -2, -1, 0, 0.5, 1, 1.5, 2])
    T_nodes = np.array([T_m4, T_m3, T_m2, T_m1, T_0, T_0p5, T_1, T_1p5, T_2])

    interpolated_function = monotonic_interpolation(logP_nodes, T_nodes)
    input_Ps = np.linspace(P_min, P_max, num=num_layers_final)

    profile = gaussian_smoothing(interpolated_function(input_Ps), sigma=0.3)

    return profile

In [ ]:
Ps = np.linspace(-4, 2.5, num=71)
Ts = piette(697, 769, 802, 914, 967, 968, 1021, 1096, 1102, 1119)

In [ ]:
import chunks_as_classes as cac
from pathlib import Path
from matplotlib import pyplot as plt

In [ ]:
test_data_file = Path("/Users/arthur/Documents/Astronomy/2019/Retrieval/Code/Data/mock-L/mock-L.2022-12-08.forward-model.PLO.JHK.noised.dat")
band_names = ["J", "H", "Ks"]

In [ ]:
JHK_data = cac.read_APOLLO_data(test_data_file, band_names)

In [ ]:
waves = JHK_data["J"].wavelengths[1:]

In [ ]:
waves

In [ ]:
opacities_interpd = opacity_Dataset.interp(
    pressures=Ps,
    temperatures=Ts,
    wavelengths=waves,
    assume_sorted=True
)

In [ ]:
opacities_interpd

In [ ]:
nc_interpd_save_file = "nir_opacities_test_interp_to_data.nc"
opacities_interpd.to_netcdf(nc_interpd_save_file)